In [1]:
#LIBRERIAS DEL PROYECTO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Este es driver que va a navegar.
driver = webdriver.Chrome()

# Función para iniciar sesión
arancia_url = 'http://arancia-001-site1.btempurl.com/'

driver.get(arancia_url)
time.sleep(3)

# Entrar a la web
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button3")
element.click()

In [2]:
import pandas as pd
import string
import numpy as np

data = pd.read_excel(r'C:\Users\Francesc\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#FECHA
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%d/%m/%Y')
data['Fecha'] = data['Fecha'].dt.strftime('%m/%d/%Y')

#TIPO DE FACTURA
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Punto de Venta'] + '-' + data['Número Desde']

# PROVEEDOR
data['Denominación Emisor'] = data['Denominación Emisor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Proveedor'] = data['Denominación Emisor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Emisor'].astype(str)

# TIPO DE CAMBIO
columnas = ['Imp. Neto Gravado', 'Imp. Neto No Gravado', 
            'Imp. Op. Exentas', 'Otros Tributos', 'IVA']
for col in columnas:
    data[col] = data[col] * data['Tipo Cambio']

# IVA (10.5 O 21)
data['% IVA'] = (data['IVA'] / data['Imp. Neto Gravado']) * 100
data['% IVA'] = np.where(abs(data['% IVA'] - 21) < 0.01, 21, data['% IVA'])
data['% IVA'] = np.where(abs(data['% IVA'] - 10.5) < 0.01, 10.5, data['% IVA'])

# CREAR COLUMNAS DE VALORES
data['NETO 10.5'] = np.nan
data['IVA 10.5'] = np.nan
data['NETO 21'] = np.nan
data['IVA 21'] = np.nan
data['NO GRAVADO'] = np.nan

# PONER 0 A TODAS LAS COLUMNAS
columnas = ['NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO']
for col in columnas:
    data[col] = data[col].fillna(0)

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)


# Asignamos los valores correspondientes a las nuevas columnas
data.loc[data['% IVA'] == 10.5, 'NETO 10.5'] = data['Imp. Neto Gravado'].astype(str)
data.loc[data['% IVA'] == 10.5, 'IVA 10.5'] = data['IVA'].astype(str)
data.loc[data['% IVA'] == 21, 'NETO 21'] = data['Imp. Neto Gravado'].astype(str)
data.loc[data['% IVA'] == 21, 'IVA 21'] = data['IVA'].astype(str)
data[col] = data[col].fillna(0)
# NO GRAVADO
data['NO GRAVADO'] = (data['Imp. Neto No Gravado'] + data['Imp. Op. Exentas'] + data['Otros Tributos'])

# PONER EN NEGATICO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    data.loc[data['Tipo2'] == 'Crédito', col] = -data[col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','Tipo3','Factura',
        'Proveedor','CUIT',
        'NETO 10.5', 'IVA 10.5',
        'NETO 21', 'IVA 21', 'NO GRAVADO']

clean_data = data[clean]

In [3]:
driver.switch_to.frame('marco')

In [4]:

for index, row in clean_data.iterrows():
    # Encuentra cada campo por su id y llena el valor
    driver.find_element(By.ID, 'DetailsView1_TextBox1').send_keys(row['Fecha'])
    driver.find_element(By.ID, 'DetailsView1_TextBox2').send_keys(row['Fecha'])
    driver.find_element(By.NAME, 'DetailsView1$ctl02').send_keys(row['Tipo3'])
    driver.find_element(By.NAME, 'DetailsView1$ctl03').send_keys(row['Factura'])
    driver.find_element(By.NAME, 'DetailsView1$ctl04').send_keys(row['Proveedor'])
    driver.find_element(By.NAME, 'DetailsView1$ctl05').send_keys(row['CUIT'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox3').send_keys(row['NETO 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox4').send_keys(row['NETO 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox5').send_keys(row['IVA 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox6').send_keys(row['IVA 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox7').send_keys(row['NO GRAVADO'])
    # Haz clic en el botón "Agregar"
    driver.find_element(By.LINK_TEXT, 'Agregar').click()
    # Espera un poco para que la página se actualice
    time.sleep(3)

In [5]:
os.remove(r'C:\Users\Francesc\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx')